# Biblioteca Musical - Atividade de Validação de Conhecimentos

A ideia é explorar os recursos apreendidos durante os cursos feitos até então e ter um projeto
base de referência para projetos futuros.

In [ ]:
#imports
#!pip install graphviz
import pandas as pd
#!pip install folium
import squarify
import folium
import seaborn as srn
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
#print(plt.style.available)
#Treinando um algoritmo
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
import graphviz
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#carregando dados como dataframe

album = pd.read_csv('ALBUM.csv')

In [ ]:
#Verificando nulos
albumNulos = pd.isnull(album["ANO"])
albumNulos

In [ ]:
album.head()

In [ ]:
artista = pd.read_csv('ARTISTA.csv')
artista.head()

In [ ]:
#removendo valores NA
mediana = album['ANO'].median()
mediana

In [ ]:
#opitei por usar a mediana para substuir valores NA na coluna ANO // Procurar como substituir só na coluna
album = album.fillna(value = mediana)
album

In [ ]:
#Convertendo a coluna ano para inteiro
album['ANO'] = album['ANO'].astype(int)

In [ ]:
#fazendo o merge dos dois dataframes
merge = album.merge(artista, on='ID_ARTISTA', how = 'outer')

merge.rename(columns={'NOME_x': 'NOME_ALBUM', 
                           'NOME_y': 'NOME_BANDA'}, inplace=True)

merge.to_csv("MERGE.csv")

In [ ]:
#resumo dos dataframe artista
artista['QTD_MEMBROS'].describe()

In [ ]:
#resumo dos dataframe album
album.describe()

In [ ]:
#testando o uso de slices 
album[10:35]

In [ ]:
album[album['ANO'] > 2000]

In [ ]:
#Procurar como usar duas condições no slice
album[album['MEDIA_SPUT'] > 4.5]

In [ ]:
#Localizando dados no dataframe
album.loc[[1]]

In [ ]:
album.loc[[1,2]]

In [ ]:
album.loc[1:8]

In [ ]:
#Traz do primeiro ao quinto dado da última coluna
album.iloc[0:5,-1]

In [ ]:
#Utilizando a função Unique - Retorna valores únicos
album['MINHA_NOTA'].unique()

In [ ]:
#Ordenando pelas maiores notas do Sputinik
albumOrdenadoNotaSput = album.sort_values(by='MEDIA_SPUT',ascending=False)
albumOrdenadoNotaSput

In [ ]:
#Usando value counts - Mostra a fequência de cada elemento
print('Quantidade de Faixas')
album['QTD_FAIXAS'].value_counts()


In [ ]:
#album.groupby(['QTD_FAIXAS']).mean()
#album.groupby(level=0).mean()
#É possível ver as estatístias da minha nota por artista
merge.groupby(['NOME_BANDA'])['MINHA_NOTA'].agg(['max','min','mean','median'])

In [ ]:
#É possível ver as estatístias da media sput por artista
merge.groupby(['NOME_BANDA'])['MEDIA_SPUT'].agg(['max','min','mean','median'])

In [ ]:
#Substituindo valores com função map
albumMap = album
banda = {1:'Black Sabbath', 2:'Iron Maiden'}
albumMap['ID_ARTISTA'].map(banda).head()

In [ ]:
#Função Pivot - Ainda preciso entender melhor como utilizar
merge.pivot(index='ID_ALBUM', columns='PAIS', values='MINHA_NOTA')


In [ ]:
artista.groupby(['GENERO']).count() 


In [ ]:
#Qual gênero mais ouvido por artista, desta vez separado pelos 3 maiores
#Pensar numa ideia de mostrar menos itens

values = artista['GENERO'].value_counts().keys().tolist()
counts = artista['GENERO'].value_counts().tolist()

othersValues = 'Others'
othersCounts = counts[3:]

values = values[0:3]
counts = counts[0:3]

counts.append(sum(othersCounts))
values.append(othersValues)


plt.style.use('seaborn-whitegrid')
fig = plt.figure(figsize =(16, 10)) 
plt.title("Gênero por Artista")
plt.pie(counts, labels = values, autopct='%.2f%%', shadow = True) 

plt.show() 

In [ ]:
#Donut Chart
figs = plt.figure(figsize = (16, 8))

my_circle = plt.Circle((0, 0), 0.7, color='white')

d = plt.pie(counts, labels=values, autopct='%1.1f%%',
            startangle=45, labeldistance=1.25)
plt.axis('equal')
plt.gca().add_artist(my_circle)
            
plt.show()

In [ ]:
#Qual gênero mais ouvido por álbum
#Gráfico de barras horizontal devido a quantidade de itens
label = merge['GENERO'].value_counts().keys().tolist()
freq = merge['GENERO'].value_counts().tolist()


#plot do gráfico
#Funcão reverse inverte a lista, permitindo que os maiores valores fiquem acima no gráfico
freq.reverse()
label.reverse()

#Faz o slicing dos últimos 7 elementos
freq = freq[-7:]
label = label[-7:]

print(freq)
print('\n\n', label)

figs = plt.figure(figsize = (16, 8))
plt.barh(label, freq)
plt.xticks(rotation = 70, fontsize = 25)
plt.yticks(fontsize = 20)

plt.show()


In [ ]:
freq.reverse()
label.reverse()

colors = ['#4287f5','#7794fc','#6aa2fc','#98befa','#2575f5', '#4287f5', '#98befa']

fig = go.Figure(go.Funnelarea(text = label, values = freq))

fig.update_layout(title_text="Gêneros Mais Ouvidos por Álbum", autosize = False, width = 800, height = 800, showlegend = False, title_x = 0.5)

fig.update_traces(textfont_size = 20, marker = dict(colors = colors))                 
                  

fig.show()

In [ ]:
#Gráfico de Mapa
#Manipulando dados para gerar o mapa

#Inserindo a longitude e a latitude ao dataframe trabalhado
paises = pd.read_csv('latitude_longitude.csv', usecols = [0, 1, 2, 3])
paises.rename(columns={'country_code': 'PAIS'}, inplace=True)
#paises['PAIS'].replace('UK', 'GB')
merge.replace('UK', 'GB')
merge = merge.merge(paises, on='PAIS', how = 'left')

#Verifica se algum valor está faltndo
merge.isnull().values.any()
merge

In [ ]:
from folium import plugins
mapa = folium.Map(location = [-0, -0], zoom_start = 1.)
coordenadas = []
for lat, lng in zip(merge.latitude.values, merge.longitude.values):
    coordenadas.append([lat, lng])   
#coordenadas

mapa.add_child(plugins.HeatMap(coordenadas))

In [ ]:

mergeP = merge.groupby(['country']).count()
mergeP = mergeP.iloc[:, [0]]
mergeP.rename(columns = {'ID_ALBUM': 'COUNT'}, inplace = True)
teste = ['Australia','Canada',  ]
mergeP['index1'] = mergeP.index
mergeP['index1'] = mergeP['index1'].astype(str)

mergeP.replace('United States', "United States of America", inplace = True)


mergeP

In [ ]:
# Gráfico choropleth

#merge['PAIS'] = merge['PAIS'].replace(['PAIS'],'CAN')


url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'


m = folium.Map(location=[-0, -0], zoom_start = 1.5)

folium.Choropleth(
    geo_data = country_shapes,
    name = "ip_region",
    data = mergeP,
    columns = ["index1", "COUNT"],
    key_on = 'feature.properties.name',
    fill_color = "YlGn",
    nan_fill_color = 'white',
    fill_opacity = 0.7,
    line_opacity = 0.4,
    legend_name = "Choropleth Álbuns",
).add_to(m)

folium.LayerControl().add_to(m)

mergeP


m

## Utilizando um Treemap

In [ ]:
artistaP = artista.groupby(['PAIS']).count()
artistaP = artistaP.iloc[:, [0]]

colors = ["red","green","blue", "grey", "yellow", "purple"]

plt.figure(figsize = (16,11), dpi = 80)
squarify.plot(sizes = artistaP.values, label = artistaP.index, color = colors, alpha = .7, text_kwargs={'fontsize' : 25})


plt.title("Álbuns por País", fontsize = 25, pad = 30.)
plt.axis('off')
plt.show()

Revisar o gráfico de linhas, fazer com a função do pandas gerou o mesmo problema 

In [ ]:
#Albuns por ano


#plt.xlabel("Ano")  
#plt.ylabel("Quantidade de Álbuns") 
fig = plt.figure(figsize =(16, 10)) 

plt.yticks(fontsize = 20)
plt.xticks(fontsize = 20)
plt.title("Quantidade de Álbuns por Ano")
agrupado = merge.groupby(['ANO']).size()
#agrupado.plot.line(color = 'blue')
srn.lineplot(data = agrupado)

Pensar numa forma melhor de plotar as médias das notas

In [ ]:
#Albuns por ano
label = merge['NOME_ALBUM'].values
freqM = merge['MINHA_NOTA'].values
freqS = merge['MEDIA_SPUT'].values

#Esse fica consideravelmente melhor no Power BI


#Por algum motivo adicionar uma vírgula faz a legenda funcionar

fig = plt.figure( figsize=(70, 18)) 
plt.yticks(fontsize = 20)
plt.xticks(fontsize = 20)
ax1, = plt.plot(label, freqM, marker = 'o') 
ax2, = plt.plot(label, freqS, marker = 'o') 

#Resolvendo o plot das linhas médias
plt.hlines(y = freqM.mean(), xmin = 0, xmax = 100, color = 'g')
plt.hlines(y = freqS.mean(), xmin = 0, xmax = 100, color = 'r')

#plt.plot(label, freqM.mean()) 

plt.legend([ax1, ax2], ['Notas Média','Notas Sputinik','Minha Média', 'Média Sputink'])
plt.xlabel("Ano")  
plt.ylabel("Quantidade de Álbuns")  
plt.xticks(rotation = 75)
plt.title("Quantidade de Álbuns por Ano")  
plt.show() 

In [ ]:
#Resumo dos dados usados a seguir
print('Minhas Notas: ')
merge['MINHA_NOTA'].describe()

In [ ]:
#Histograma das minhas notas
#Histograma e Boxplot são mt melhores por aqui!!
#Ficaram iguais ao do Power BI

#merge.hist(column = 'MINHA_NOTA', bins = 5)
#plt.show()
#Vai ser descontinuado 
fig = plt.figure( figsize=(12.8, 9.6)) 
plt.yticks(fontsize = 20)
plt.xticks(fontsize = 15)

srn.histplot(data =merge['MINHA_NOTA'], kde = True, bins = 8).set_title('Minha Nota')

In [ ]:
#Histograma das notas Sputinik


#merge.hist(column = 'MEDIA_SPUT', bins = 5)
#plt.show()
fig = plt.figure( figsize=(12.8, 9.6)) 
plt.yticks(fontsize = 20)
plt.xticks(fontsize = 15)

srn.histplot(data =merge['MEDIA_SPUT'], kde = True, bins = 8).set_title('Média Sputinik')

Revisar os boxplots

In [ ]:
fig = plt.figure( figsize=(12.8, 9.6)) 
plt.yticks(fontsize = 15)
plt.xticks(fontsize = 10)

merge.boxplot(column='MINHA_NOTA')


In [ ]:
#Boxplot das minhas notas
fig = plt.figure( figsize=(12.8, 9.6)) 
plt.yticks(fontsize = 15)
plt.xticks(fontsize = 10)

srn.boxplot(x = merge['MINHA_NOTA'], y = merge['NOME_BANDA']).set_title('Minha Nota')

In [ ]:
#Boxplot das notas sputinik
fig = plt.figure( figsize=(12.8, 9.6)) 
plt.yticks(fontsize = 15)
plt.xticks(fontsize = 10)

merge.boxplot(column='MEDIA_SPUT')

In [ ]:
#Boxplot das notas sputinik
fig = plt.figure( figsize=(12.8, 9.6)) 
plt.yticks(fontsize = 15)
plt.xticks(fontsize = 10)

srn.boxplot(x =merge['MEDIA_SPUT'], y = merge['NOME_BANDA']).set_title('Nota Sputinik')

In [ ]:
merge.corr()

In [ ]:
#Correlação
fig = plt.figure( figsize=(12.8, 9.6)) 
plt.yticks(fontsize = 15)
plt.xticks(fontsize = 10)

size = 15
corr = merge.corr()    
plot = srn.heatmap(corr, annot = True, fmt=".2f", linewidths=.6)
plot

In [ ]:
# Seleção de variáveis preditoras (Feature Selection)
atributos = ['ID_ALBUM', 'TEMPO_DURACAO', 'QTD_FAIXAS', 'MEDIA_SPUT', 'ID_ARTISTA', 'ANO', 'QTD_MEMBROS']

# Variável a ser prevista
atrib_prev = ['MINHA_NOTA']

# Criando objetos
X = merge[atributos].values
Y = merge[atrib_prev].values
Y=Y.astype('int')

# Definindo a taxa de split
split_test_size = 0.30
#X
#Y
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = split_test_size, random_state = 42)

In [ ]:
#Imprimindo os resultados
print("{0:0.2f}% nos dados de treino".format((len(X_treino)/len(merge.index)) * 100))
print("{0:0.2f}% nos dados de teste".format((len(X_teste)/len(merge.index)) * 100))

In [ ]:
#Construindo e treinando o modelo
# Utilizando um classificador Naive Bayes
from sklearn.naive_bayes import GaussianNB

# Criando o modelo preditivo
modelo_v1 = GaussianNB()

# Treinando o modelo
modelo_v1.fit(X_treino, Y_treino.ravel())

In [ ]:
#Previsões
previsoes = modelo_v1.predict(X_teste)
previsoes

In [ ]:
#Verificando a exatidão do modelo de treino

from sklearn import metrics

predict_train = modelo_v1.predict(X_treino)

print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, predict_train)))
print()

In [ ]:
#verificando exatidão no modelo de teste
predict_test = modelo_v1.predict(X_teste)

print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, predict_test)))
print()

In [ ]:
#salvando o modelo
import pickle
filename = 'moelo_treinado.sav'
pickle.dump(modelo_v1, open(filename, 'wb'))

In [ ]:
#Utilizando o modelo
loaded_model = pickle.load(open(filename, 'rb'))
resultado1 = loaded_model.predict(X_teste[18].reshape(1, -1))
resultado2 = loaded_model.predict(X_teste[18].reshape(1, -1))

#print(Y_teste)
print(resultado1)
print(resultado2)

In [ ]:
#Selecionando os Atributos em busca de melhorar a precisão do modelo
#Utilizando do algoritmo Extra Trees Classfifier para descobrir a importância dos atributos
floresta = ExtraTreesClassifier()
floresta.fit(X_treino, Y_treino.ravel())
importancias = floresta.feature_importances_
importancias

In [ ]:
#Usando subconjunto dos mais relevantes para os dados de treino
X_treino2 = X_treino[:, [0,1,2,3,5]]
X_teste2 = X_teste[:, [0,1,2,3,5]]

In [ ]:
modelo_v2 = GaussianNB()
modelo_v2.fit(X_treino2, Y_treino.ravel())
previsoes2 = modelo_v2.predict(X_teste2)
taxa_acerto = accuracy_score(Y_teste, previsoes2)
taxa_acerto

# Resultado
Apesar da seleção dos atributos, o modelo V2 não é melhor que o modelo V1. 
## O que fazer agora? 

Buscar novos agoritmos ou Aumentar o volume de dados no dataset. 

Todas as variáveis usadas são numéricas, então o tratamento de categóricas não é necessário.

In [ ]:
#Testando a criação do modelo com árvore de decisão
arvore = DecisionTreeClassifier()
arvore.fit(X_treino2, Y_treino.ravel())

In [ ]:
#Exportação da Árvore de Decisão .dot
export_graphviz(arvore, out_file = 'tree.dot')

In [ ]:
previsoes2 = arvore.predict(X_teste2)
previsoes2

In [ ]:
confusao = confusion_matrix(Y_teste, previsoes2)
confusao

In [ ]:
taxa_acerto = accuracy_score(Y_teste, previsoes2)
taxa_acerto

In [ ]:
#Random Forest
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treino2, Y_treino.ravel())

In [ ]:
previsoes3 = floresta.predict(X_teste2)
previsoes3

In [ ]:
consfusao = confusion_matrix(Y_teste, previsoes3)
confusao

In [ ]:
# Accuracy nos dados de treino
# Ovefittin? 
predict_train = floresta.predict(X_treino2)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, predict_train)))
print()

In [ ]:
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, previsoes3)))
print()